In [1]:
from sklearn.datasets import fetch_openml
import numpy as np
import time
import torch
from scipy.optimize import linprog
from qpsolvers import solve_qp
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
from torch.autograd import Function
import torch.nn as nn
from sklearn.model_selection import train_test_split
import sys
import StochasticGhost
import importlib

2023-12-20 11:15:39.564826: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 11:16:09.654869: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
/home/harsh/anaconda3/envs/ghost/lib/python3.10/site-packages/ot/backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


In [2]:
xd, yd = fetch_openml('mnist_784', version=1, return_X_y=True)
xd = (xd/255).astype('float32')

/home/harsh/anaconda3/envs/ghost/lib/python3.10/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [14]:
class CustomNetwork(nn.Module):

    # For now the input data is passed as init parameters
    def __init__(self, layer_sizes, itrain, otrain, ival, oval):
        super(CustomNetwork, self).__init__()

        # Create a list of linear layers based on layer_sizes
        self.itrain = itrain
        self.otrain = otrain
        self.ival = ival
        self.oval = oval
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))

    def forward(self, x):
        # self.input = x
        for layer in self.layers[:-1]:
            x = torch.sigmoid(layer(x))
        x = self.layers[-1](x)
        return x

    def compute_loss(self, Y, Y_hat):
        """
        compute loss function
        """
        # L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
        L_sum = 0.5*torch.sum(torch.square(Y - Y_hat))

        m = Y.shape[0]
        L = (1./m) * L_sum

        return L

    def obj_fun(self, params, minibatch):
        model_parameters = list(self.parameters())
        # with torch.no_grad():
        x = self.itrain
        y = self.otrain
        samples = np.random.choice(len(y), minibatch, replace=False)
        for i in range(len(params)):
            model_parameters[i].data = torch.Tensor(params[i])
        # self.compute_loss(fval, y[minibatch])
        fval = self.compute_loss(self.forward(x[samples, :]), y[samples])
        return fval.item()

    def obj_grad(self, params, minibatch):
        # f_grad = {}
        fgrad = []
        x = self.itrain
        y = self.otrain
        samples = np.random.choice(len(y), minibatch, replace=False)
        # with torch.no_grad():
        for name, param in self.named_parameters():
            fgrad.append(torch.autograd.grad(
                self.compute_loss(self.forward(x[samples, :]), y[samples]), param, retain_graph=True)[0].view(-1))
        fgrad = torch.cat(fgrad, dim=0)
        return fgrad

    def conf(self, params, minibatch):
        conf_val = None
        #x_val = self.ival
        #y_val = self.oval
        #samples = np.random.choice(len(y_val), minibatch, replace=False)
        params = list(self.parameters())

# Extract the weights and biases
        W1 = params[0].reshape(-1)
        W2 = params[2].reshape(-1)
        B1 = params[1].reshape(-1)
        B2 = params[3].reshape(-1)

# Perform the operations in PyTorch
        conf_val = torch.dot(W1, W1) / 4. + torch.dot(W2, W2) / 4. + torch.dot(B1, B1) + torch.dot(B2, B2)
        """
        TODO: Compute
        
        """
        return conf_val.item() - 5.

    def conJ(self, params, minibatch):
        # x_val = self.ival
        # y_val = self.oval
        # samples = np.random.choice(len(y_val), minibatch, replace=False)
        # with torch.no_grad():
        #for name, param in self.named_parameters():
        # print(name)
        params = list(self.parameters())
        cgrad = [(params[0]/2.).reshape(-1), 2*params[1].reshape(-1), (params[2]/2.).reshape(-1), 2*params[3].reshape(-1)]

        cgrad = torch.cat(cgrad, dim=0)
        print(cgrad)
        print(cgrad.shape)
        """"
        TODO: Compute

        """
        return cgrad

In [4]:
yd = yd.astype(int)
y = np.zeros((yd.shape[0],))
y[np.where(yd == 1)] = 1

In [5]:
def paramvals(maxiter, beta, rho, lamb, hess, tau, mbsz, numcon, geomp, stepdecay, gammazero, zeta, N, n):
    params = {
        'maxiter': maxiter,  # number of iterations performed
        'beta': beta,  # trust region size
        'rho': rho,  # trust region for feasibility subproblem
        'lamb': lamb,  # weight on the subfeasibility relaxation
        'hess': hess,  # method of computing the Hessian of the QP, options include 'diag' 'lbfgs' 'fisher' 'adamdiag' 'adagraddiag'
        'tau': tau,  # parameter for the hessian
        'mbsz': mbsz,  # the standard minibatch size, used for evaluating the progress of the objective and constraint
        'numcon': numcon,  # number of constraint functions
        'geomp': geomp,  # parameter for the geometric random variable defining the number of subproblem samples
        # strategy for step decrease, options include 'dimin' 'stepwise' 'slowdimin' 'constant'
        'stepdecay': stepdecay,
        'gammazero': gammazero,  # initial stepsize
        'zeta': zeta,  # parameter associated with the stepsize iteration
        'N': N,  # Train/val sample size
        'n': n  # Total number of parameters
    }
    return params

In [6]:
import types

In [7]:
def reload_package(root_module):
    package_name = root_module.__name__

    # get a reference to each loaded module
    loaded_package_modules = dict([
        (key, value) for key, value in sys.modules.items()
        if key.startswith(package_name) and isinstance(value, types.ModuleType)])

    # delete references to these loaded modules from sys.modules
    for key in loaded_package_modules:
        del sys.modules[key]

    # load each of the modules again;
    # make old modules share state with new modules
    for key in loaded_package_modules:
        print("Loading ", key)
        newmodule = __import__(key)
        oldmodule = loaded_package_modules[key]
        oldmodule.__dict__.clear()
        oldmodule.__dict__.update(newmodule.__dict__)

In [8]:
import StochasticGhost
reload_package(StochasticGhost)

Loading  StochasticGhost


In [15]:
maxiter = 1
layer_sizes = [70, 15, 1]
X_train, X_val, Y_train, Y_val = train_test_split(
    torch.tensor(xd.values), torch.tensor(y), test_size=0.3, random_state=42)
train_loss_list = []
val_loss_list = []
cons_bound = 10.
# X_train = torch.tensor()
net = CustomNetwork(
    layer_sizes, X_train[:, :70], Y_train, X_val[:, :70], Y_val)
nn_parameters = list(net.parameters())
initw = [param.data for param in nn_parameters]
num_param = sum(p.numel() for p in net.parameters())
# net.train_and_update(xd, yd)
# output_t = net.forward(X_train[:, :70])
# output_v = net.forward(X_val[:, :70])
# obj = net.compute_loss(Y_train[:70], output_t)
# constraint = output_v-cons_bound
params = paramvals(maxiter=2, beta=10, rho=0.8, lamb=0.5, hess='diag', tau=1., mbsz=1,
                   numcon=1, geomp=0.7, stepdecay='dimin', gammazero=0.1, zeta=0.1, N=X_val.shape[0], n=num_param)
w, iterfs, itercs = StochasticGhost.StochasticGhost(
    net.obj_fun, net.obj_grad, [net.conf], [net.conJ], initw, params)

LOOKIE LOOKIE:::::::::  -3.5958938598632812
<class 'numpy.ndarray'>
tensor([ 0.0316,  0.0244, -0.0472,  ...,  0.0969, -0.1056, -0.0298],
       grad_fn=<CatBackward0>)
torch.Size([1081])
<class 'numpy.ndarray'>
<class 'float'> <class 'numpy.ndarray'> <class 'numpy.float64'> <class 'numpy.ndarray'>
[-0.         -0.         -0.         ... -0.0035899  -0.00339783
 -0.00687086]
[ 0.03157401  0.02438785 -0.04720455 ...  0.09693245 -0.10561271
 -0.02977518]
<class 'numpy.ndarray'>
tensor([ 0.0316,  0.0244, -0.0472,  ...,  0.0969, -0.1056, -0.0298],
       grad_fn=<CatBackward0>)
torch.Size([1081])
<class 'numpy.ndarray'>
<class 'float'> <class 'numpy.ndarray'> <class 'numpy.float64'> <class 'numpy.ndarray'>
[ 0.          0.          0.         ... -0.0071798  -0.00679565
 -0.01374172]
[ 0.03157401  0.02438785 -0.04720455 ...  0.09693245 -0.10561271
 -0.02977518]
<class 'numpy.ndarray'>
tensor([ 0.0316,  0.0244, -0.0472,  ...,  0.0969, -0.1056, -0.0298],
       grad_fn=<CatBackward0>)
torch.

/home/harsh/anaconda3/envs/ghost/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:38: UserWarning: Converted P to scipy.sparse.csc.csc_matrix
For best performance, build P as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/harsh/anaconda3/envs/ghost/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:38: UserWarning: Converted G to scipy.sparse.csc.csc_matrix
For best performance, build G as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
/home/harsh/anaconda3/envs/ghost/lib/python3.10/site-packages/qpsolvers/conversions/ensure_sparse_matrices.py:38: UserWarning: Converted A to scipy.sparse.csc.csc_matrix
For best performance, build A as a scipy.sparse.csc_matrix rather than as a numpy.ndarray
  warnings.warn(
